In [ ]:
import math
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Lambda
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Activation

from sklearn import preprocessing
from sklearn import utils
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor


from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/gdrive/My Drive/Bitcoin1DHistoricalData.csv')
data

,Date,Price,Open,High,Low,Vol.,Change %
0,"May 01, 2021","57,807.1","57,719.1","58,449.4","57,029.5",63.41K,0.15%
1,"Apr 30, 2021","57,720.3","53,562.3","57,925.6","53,088.7",103.74K,7.77%
2,"Apr 29, 2021","53,560.8","54,838.6","55,173.7","52,400.0",83.90K,-2.34%
3,"Apr 28, 2021","54,841.4","55,036.0","56,419.9","53,876.4",86.96K,-0.35%
4,"Apr 27, 2021","55,036.5","54,011.1","55,427.8","53,345.0",84.08K,1.88%
...,...,...,...,...,...,...,...
3936,"Jul 22, 2010",0.1,0.1,0.1,0.1,2.16K,0.00%
3937,"Jul 21, 2010",0.1,0.1,0.1,0.1,0.58K,0.00%
3938,"Jul 20, 2010",0.1,0.1,0.1,0.1,0.26K,0.00%
3939,"Jul 19, 2010",0.1,0.1,0.1,0.1,0.57K,0.00%


16)

In [ ]:
def mdy_to_ymd(d):
    return datetime.strptime(d, '%b %d, %Y').strftime('%Y-%m-%d')

for i in range(0, 3941):
  data['Date'][i] = mdy_to_ymd(data['Date'][i])

data['Date']

0       2021-05-01
1       2021-04-30
2       2021-04-29
3       2021-04-28
4       2021-04-27
           ...    
3936    2010-07-22
3937    2010-07-21
3938    2010-07-20
3939    2010-07-19
3940    2010-07-18
Name: Date, Length: 3941, dtype: object

In [ ]:
data = data.apply(lambda x: x.str.replace(',', ''))
data['Vol.'] = data['Vol.'].str.replace(r'\D','')
data

,Date,Price,Open,High,Low,Vol.,Change %
0,2021-05-01,57807.1,57719.1,58449.4,57029.5,6341,0.15%
1,2021-04-30,57720.3,53562.3,57925.6,53088.7,10374,7.77%
2,2021-04-29,53560.8,54838.6,55173.7,52400.0,8390,-2.34%
3,2021-04-28,54841.4,55036.0,56419.9,53876.4,8696,-0.35%
4,2021-04-27,55036.5,54011.1,55427.8,53345.0,8408,1.88%
...,...,...,...,...,...,...,...
3936,2010-07-22,0.1,0.1,0.1,0.1,216,0.00%
3937,2010-07-21,0.1,0.1,0.1,0.1,058,0.00%
3938,2010-07-20,0.1,0.1,0.1,0.1,026,0.00%
3939,2010-07-19,0.1,0.1,0.1,0.1,057,0.00%


In [ ]:
data['High'] = data['High'] = pd.to_numeric(data['High'], errors='coerce')
data['Open'] = data['Open'] = pd.to_numeric(data['Open'], errors='coerce')
data['Low'] = data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
data['Vol.'] = data['Vol.'] = pd.to_numeric(data['Vol.'], errors='coerce')
data['Price'] = data['Price'] = pd.to_numeric(data['Price'], errors='coerce')
data

,Date,Price,Open,High,Low,Vol.,Change %
0,2021-05-01,57807.1,57719.1,58449.4,57029.5,6341.0,0.15%
1,2021-04-30,57720.3,53562.3,57925.6,53088.7,10374.0,7.77%
2,2021-04-29,53560.8,54838.6,55173.7,52400.0,8390.0,-2.34%
3,2021-04-28,54841.4,55036.0,56419.9,53876.4,8696.0,-0.35%
4,2021-04-27,55036.5,54011.1,55427.8,53345.0,8408.0,1.88%
...,...,...,...,...,...,...,...
3936,2010-07-22,0.1,0.1,0.1,0.1,216.0,0.00%
3937,2010-07-21,0.1,0.1,0.1,0.1,58.0,0.00%
3938,2010-07-20,0.1,0.1,0.1,0.1,26.0,0.00%
3939,2010-07-19,0.1,0.1,0.1,0.1,57.0,0.00%


In [ ]:
data = data.replace(np.nan, 0)
data

,Date,Price,Open,High,Low,Vol.,Change %
0,2021-05-01,57807.1,57719.1,58449.4,57029.5,6341.0,0.15%
1,2021-04-30,57720.3,53562.3,57925.6,53088.7,10374.0,7.77%
2,2021-04-29,53560.8,54838.6,55173.7,52400.0,8390.0,-2.34%
3,2021-04-28,54841.4,55036.0,56419.9,53876.4,8696.0,-0.35%
4,2021-04-27,55036.5,54011.1,55427.8,53345.0,8408.0,1.88%
...,...,...,...,...,...,...,...
3936,2010-07-22,0.1,0.1,0.1,0.1,216.0,0.00%
3937,2010-07-21,0.1,0.1,0.1,0.1,58.0,0.00%
3938,2010-07-20,0.1,0.1,0.1,0.1,26.0,0.00%
3939,2010-07-19,0.1,0.1,0.1,0.1,57.0,0.00%


In [ ]:
# Data Spliting
X = data.drop(['Price', 'Date', 'Change %'], axis = 1)

y = data['Price']

X_test, X_train, y_test, y_train = train_test_split(X,y, test_size = 0.9699 ,random_state = 42, shuffle = False)
X_train

,Open,High,Low,Vol.
118,32192.9,34755.9,32029.6,15521.0
119,29359.7,33233.5,29008.0,24087.0
120,28951.7,29627.1,28712.4,10090.0
121,28866.8,29298.8,28025.0,13669.0
122,27374.5,28979.3,27374.5,15982.0
...,...,...,...,...
3936,0.1,0.1,0.1,216.0
3937,0.1,0.1,0.1,58.0
3938,0.1,0.1,0.1,26.0
3939,0.1,0.1,0.1,57.0


17)

In [ ]:
lin_regression = LinearRegression().fit(X_train, y_train)
lin_pred = lin_regression.predict(X_test)
lin_pred_score = math.sqrt(mean_squared_error(y_test, lin_pred))
print('test score : %.2f RMSE' % (lin_pred_score))

test score : 961.10 RMSE


In [ ]:
ridge = Ridge().fit(X_train, y_train)
ridge_pred = ridge.predict(X_test)
ridge_pred_score = math.sqrt(mean_squared_error(y_test, ridge_pred))
print('test score : %.2f RMSE' % (ridge_pred_score))

test score : 961.10 RMSE


In [ ]:
lasso = Lasso().fit(X_train, y_train)
lasso_pred = lasso.predict(X_test)
lasso_pred_score = math.sqrt(mean_squared_error(y_test, lasso_pred))
print('test score : %.2f RMSE' % (lasso_pred_score))

test score : 994.15 RMSE


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 21548405.98461891, tolerance: 7675557.324591758
  positive)


In [ ]:
svr_poly = SVR(kernel='poly').fit(X_train,y_train)
svr_poly_pred = svr_poly.predict(X_test)
svr_poly_pred_score = math.sqrt(mean_squared_error(y_test, svr_poly_pred))
print('test score : %.2f RMSE' % (svr_poly_pred_score))

test score : 19534.72 RMSE


In [ ]:
svr_rbf = SVR(kernel='rbf', gamma=0.1).fit(X_train, y_train)
svr_rbf_pred = svr_rbf.predict(X_test)
svr_rbf_pred_score = math.sqrt(mean_squared_error(y_test, svr_rbf_pred))
print('test score : %.2f RMSE' % (svr_rbf_pred_score))

test score : 49181.85 RMSE


In [ ]:
#LSTM Model for 17
dataset = data.values
dataset = np.delete(dataset,0,1)
dataset = np.delete(dataset,5,1)
dataset.astype('float32')

array([[5.78071e+04, 5.77191e+04, 5.84494e+04, 5.70295e+04, 6.34100e+03],
       [5.77203e+04, 5.35623e+04, 5.79256e+04, 5.30887e+04, 1.03740e+04],
       [5.35608e+04, 5.48386e+04, 5.51737e+04, 5.24000e+04, 8.39000e+03],
       ...,
       [1.00000e-01, 1.00000e-01, 1.00000e-01, 1.00000e-01, 2.60000e+01],
       [1.00000e-01, 1.00000e-01, 1.00000e-01, 1.00000e-01, 5.70000e+01],
       [1.00000e-01, 0.00000e+00, 1.00000e-01, 1.00000e-01, 8.00000e+00]],
      dtype=float32)

In [ ]:
train_size = int(len(dataset) * 0.9699)
test_size = len(dataset) - train_size
test, train = dataset[0:test_size,0:1], dataset[test_size:len(dataset),1:2]
print(len(train), len(test))
print(train.shape[1], test.shape[1])

3822 119
1 1


In [ ]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [ ]:
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
trainY.shape

(3821,)

In [ ]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
tensor_trainX = tf.convert_to_tensor(trainX, np.float32)
tensor_testX = tf.convert_to_tensor(testX, np.float32)

In [ ]:
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(tensor_trainX, trainY, epochs=100, batch_size=1, verbose=2)

In [ ]:
trainPredict = model.predict(tensor_trainX)
testPredict = model.predict(tensor_testX)

trainScore = math.sqrt(mean_squared_error(trainY, trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY, testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
trainY.shape

Train Score: 5345.90 RMSE
Test Score: 49464.87 RMSE


(3821,)

18)

In [ ]:
# voting
model_1 = LinearRegression()
model_2 = Lasso()
model_3 = Ridge()
 
# Making the final model using voting regressor
final_model = VotingRegressor(
    estimators=[('lr', model_1), ('ridge', model_2), ('lasso', model_3)])

lab_enc = preprocessing.LabelEncoder()
encoded_y_train = lab_enc.fit_transform(y_train)
final_model.fit(X_train, encoded_y_train) 

# predicting the output on the test dataset
pred_final = final_model.predict(X_test)
pred_final

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 357507082.6245327, tolerance: 320461.6265364369
  positive)


array([10484.97221027,  9337.2302848 ,  9814.86780439,  9882.33090504,
        9761.80364428,  8460.84019335,  8884.55113617,  9214.79881271,
        9120.01670407,  9416.75200264, 10067.07892262,  9861.95742838,
       10012.0820571 , 10042.494356  , 10947.79713949, 11213.9735745 ,
       11380.52799661, 11280.11090673, 10583.92140455, 10855.94245244,
       10841.83152119, 10331.28513735, 10573.46438247, 10047.5224942 ,
       10334.24605294, 10658.71729245, 10441.54273157, 10291.98243884,
       10519.10517077, 10632.83565924, 10677.58768853, 10507.12820037,
       10366.46364277,  9905.11164584, 10107.9032128 ,  9877.94679881,
        9081.29682391,  9414.08018092,  9405.24521374,  9684.56326503,
       10028.03485377, 10351.81648148, 10474.06873357, 10259.05940116,
       10510.21332957,  9945.42868391,  9875.51884786, 10229.27489972,
       10952.24060537,  9892.26138531, 10332.79718978,  9886.92761685,
        9662.17281076,  9377.23079357,  9098.59351407,  8808.19053328,
      

In [ ]:
# bagging
 
# Making the final model using bagging regressor
final_bag_model = BaggingRegressor(base_estimator=SVR(kernel='poly'),
                         n_estimators=10, random_state=0)

lab_enc = preprocessing.LabelEncoder()
encoded_y_train = lab_enc.fit_transform(y_train)
# training all the model on the train dataset
final_bag_model.fit(X_train, encoded_y_train) 

# predicting the output on the test dataset
pred_final_bag = final_bag_model.predict(X_test)
pred_final_bag

array([18836.62131811, 17754.21103289, 16478.34052699, 17384.6498943 ,
       16578.48230902, 15208.35029936, 12161.67242564, 13554.03866812,
       17709.99392177, 18584.2382099 , 18297.77169005, 18604.77252581,
       19234.5977845 , 24528.55093531, 22953.09174987, 26381.08688394,
       24474.46330512, 27204.99276679, 24677.53784575, 21933.60410447,
       20641.65183223, 21636.52659999, 19119.39179605, 17898.12999581,
       20253.56118046, 20223.54595401, 18824.2991179 , 18265.25846415,
       19789.3438922 , 20433.64128943, 19959.30371192, 20864.5642583 ,
       19938.57485887, 19225.17083603, 16917.35973814, 17013.00169814,
       15744.02716064, 16638.09868198, 18539.67550705, 16647.01950802,
       18923.97764675, 19078.04336117, 19800.45593112, 19659.78170589,
       21215.83015583, 19911.59612136, 18983.55223906, 21391.72068591,
       22269.26268031, 22187.52038631, 20134.85241419, 19781.58130149,
       19260.64805137, 16574.2236843 , 13538.42762719, 13132.09554733,
      

In [ ]:
# Boosting
 
# Making the final model using Ada Boost regressor using rbf kernel of svr
final_Boost_model = AdaBoostRegressor(base_estimator=SVR(),
                         n_estimators=10, random_state=0)

lab_enc = preprocessing.LabelEncoder()
encoded_y_train = lab_enc.fit_transform(y_train)
# training all the model on the train dataset
final_Boost_model.fit(X_train, encoded_y_train) 

# predicting the output on the test dataset
pred_final_boost = final_Boost_model.predict(X_test)
pred_final_boost

array([1572.11230608, 1572.5232    , 1572.65067573, 1572.46332793,
       1572.62627178, 1573.93707925, 1574.54175761, 1574.00936637,
       1573.93958551, 1572.92925645, 1572.38788721, 1572.44348882,
       1572.34618693, 1572.14432881, 1571.93452054, 1571.91020032,
       1571.89024736, 1571.88933251, 1571.94309956, 1571.96769602,
       1571.97535501, 1572.03516135, 1572.07166498, 1572.22405256,
       1572.17435766, 1572.04982047, 1572.07775874, 1572.14224216,
       1572.05414492, 1572.02185946, 1572.0425168 , 1572.07107659,
       1572.10398668, 1572.2891266 , 1572.33258677, 1572.43179221,
       1573.11356243, 1573.3851121 , 1572.64134832, 1572.616845  ,
       1572.25877458, 1572.14952471, 1572.05021158, 1572.12071798,
       1572.06009344, 1572.25170206, 1572.46688439, 1572.09483061,
       1571.95151303, 1572.08843926, 1572.21661526, 1572.33504452,
       1572.50513567, 1572.98259502, 1573.67909574, 1574.37866064,
       1575.24131478, 1575.8121813 , 1573.77842683, 1574.13991

19)

In [ ]:
# Making the final model using Ada Boost regressor by rbf kernel of svr and 10 weak learners
final_Boost_model1 = AdaBoostRegressor(base_estimator=SVR(kernel='rbf', gamma= 0.01),
                         n_estimators=10, random_state=0)

lab_enc = preprocessing.LabelEncoder()
encoded_y_train = lab_enc.fit_transform(y_train)
# training all the model on the train dataset
final_Boost_model1.fit(X_train, encoded_y_train) 

# predicting the output on the test dataset
pred_final_boost1 = final_Boost_model1.predict(X_test)
pred_final_boost1_score = math.sqrt(mean_squared_error(y_test, pred_final_boost1))
print('test score : %.2f RMSE' % (pred_final_boost1_score))

test score : 48611.42 RMSE


In [ ]:
final_Boost_model2 = AdaBoostRegressor(base_estimator=SVR(kernel='poly', degree = 2),
                         n_estimators=10, random_state=0)

lab_enc = preprocessing.LabelEncoder()
encoded_y_train = lab_enc.fit_transform(y_train)
final_Boost_model2.fit(X_train, encoded_y_train) 

# predicting the output on the test dataset
pred_final_boost2 = final_Boost_model2.predict(X_test)
pred_final_boost2_score = math.sqrt(mean_squared_error(y_test, pred_final_boost2))
print('test score : %.2f RMSE' % (pred_final_boost2_score))

test score : 39445.92 RMSE


In [ ]:
final_Boost_model3 = AdaBoostRegressor(base_estimator=SVR(kernel='rbf', gamma= 0.01),
                         n_estimators=20, random_state=0)

lab_enc = preprocessing.LabelEncoder()
encoded_y_train = lab_enc.fit_transform(y_train)
# training all the model on the train dataset
final_Boost_model3.fit(X_train, encoded_y_train) 

# predicting the output on the test dataset
pred_final_boost3 = final_Boost_model3.predict(X_test)
pred_final_boost3_score = math.sqrt(mean_squared_error(y_test, pred_final_boost3))
print('test score : %.2f RMSE' % (pred_final_boost1_score))

test score : 48611.42 RMSE


In [ ]:
final_Boost_model4 = AdaBoostRegressor(base_estimator=SVR(kernel='poly', degree = 2),
                         n_estimators=20, random_state=0)

lab_enc = preprocessing.LabelEncoder()
encoded_y_train = lab_enc.fit_transform(y_train)
# training all the model on the train dataset
final_Boost_model4.fit(X_train, encoded_y_train) 

# predicting the output on the test dataset
pred_final_boost4 = final_Boost_model2.predict(X_test)
pred_final_boost4_score = math.sqrt(mean_squared_error(y_test, pred_final_boost4))
print('test score : %.2f RMSE' % (pred_final_boost4_score))

test score : 39445.92 RMSE


20)

In [ ]:
# Random Forest
random_forest_model = RandomForestRegressor(max_depth=2,random_state=0)

#encode our target 
lab_enc = preprocessing.LabelEncoder()
encoded_y_train = lab_enc.fit_transform(y_train)

# train our random forest based model :
random_forest_model.fit(X_train,encoded_y_train)

# predicting the output on the test dataset
random_forest_model_pred = random_forest_model.predict(X_test)
random_forest_model_pred_score = math.sqrt(mean_squared_error(y_test, random_forest_model_pred))
print('test score : %.2f RMSE' % (random_forest_model_pred_score))

test score : 47243.62 RMSE


In [ ]:
# Random Forest (second edition with 4 layers)
random_forest_model4 = RandomForestRegressor(max_depth=4,random_state=0)

lab_enc = preprocessing.LabelEncoder()
encoded_y_train = lab_enc.fit_transform(y_train)

# train our random forest based model :
random_forest_model4.fit(X_train,encoded_y_train)

# predicting the output on the test dataset
random_forest_model4_pred = random_forest_model4.predict(X_test)
random_forest_model4_pred_score = math.sqrt(mean_squared_error(y_test, random_forest_model4_pred))
print('test score : %.2f RMSE' % (random_forest_model4_pred_score))

test score : 46942.08 RMSE


21)

In [ ]:
def ignore_5_percent_error (y_true, y_predict):
  rmspe = np.sqrt(np.mean(np.square(((y_true - y_predict) / y_true)), axis = 0))
  if rmspe > 0.95 :
    print('RMSPE value and Prediction status are ' + str(rmspe) + ' and True')
  else :
    print('RMSPE value and Prediction status are ' + str(rmspe) + ' and False')

  return   

ignore_5_percent_error(y_test, random_forest_model4_pred)

RMSPE value and Prediction status are 0.9406361017366941 and False


22)

In [ ]:
# create an empty column : Open-diff:
data22 = data 
data22['Open-diff'] = ""
data22['Open-diff'][3940] = 0

for i in range(0, 3940):
  if data22['Open'][i+1] > data22['Open'][i]:
    data22['Open-diff'][i] = 1.0
  
  elif data22['Open'][i+1] < data22['Open'][i]:
    data22['Open-diff'][i] = -1.0

  elif data22['Open'][i+1] == data22['Open'][i]:
    data22['Open-diff'][i] = 0.0

data22

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python

,Date,Price,Open,High,Low,Vol.,Change %,Open-diff
0,2021-05-01,57807.1,57719.1,58449.4,57029.5,6341.0,0.15%,-1
1,2021-04-30,57720.3,53562.3,57925.6,53088.7,10374.0,7.77%,1
2,2021-04-29,53560.8,54838.6,55173.7,52400.0,8390.0,-2.34%,1
3,2021-04-28,54841.4,55036.0,56419.9,53876.4,8696.0,-0.35%,-1
4,2021-04-27,55036.5,54011.1,55427.8,53345.0,8408.0,1.88%,-1
...,...,...,...,...,...,...,...,...
3936,2010-07-22,0.1,0.1,0.1,0.1,216.0,0.00%,0
3937,2010-07-21,0.1,0.1,0.1,0.1,58.0,0.00%,0
3938,2010-07-20,0.1,0.1,0.1,0.1,26.0,0.00%,0
3939,2010-07-19,0.1,0.1,0.1,0.1,57.0,0.00%,-1


In [ ]:

x22 = data22['Open'].values
y22 = data22['Open-diff'].values

xtest, xtrain, ytest, ytrain = train_test_split(x22, y22, test_size = 0.9699, random_state = 42, shuffle=False)
ytrain

array([-1.0, -1.0, -1.0, ..., 0.0, -1.0, 0], dtype=object)

In [ ]:
xtrain = np.reshape(xtrain, (xtrain.shape[0], 1))
xtest = np.reshape(xtest, (xtest.shape[0], 1))
ytrain = np.reshape(ytrain, (ytrain.shape[0], 1))
ytrain.shape

(3823, 1)

In [ ]:
xtrain = np.reshape(xtrain, (xtrain.shape[0], 1, xtrain.shape[1]))
xtest = np.reshape(xtest, (xtest.shape[0], 1, xtest.shape[1]))
ytrain = np.reshape(ytrain, (ytrain.shape[0], 1, ytrain.shape[1]))
ytrain.shape

(3823, 1, 1)

In [ ]:
tensor_xtrain = tf.convert_to_tensor(xtrain, np.float32)
tensor_xtest = tf.convert_to_tensor(xtest, np.float32)
tensor_ytrain = tf.convert_to_tensor(ytrain, np.float32)
tensor_ytrain.shape

TensorShape([3823, 1, 1])

In [ ]:
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(tensor_xtrain, tensor_ytrain, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
3823/3823 - 6s - loss: 0.8916
Epoch 2/100
3823/3823 - 4s - loss: 0.8904
Epoch 3/100
3823/3823 - 4s - loss: 0.8904
Epoch 4/100
3823/3823 - 4s - loss: 0.8904
Epoch 5/100
3823/3823 - 4s - loss: 0.8905
Epoch 6/100
3823/3823 - 4s - loss: 0.8905
Epoch 7/100
3823/3823 - 4s - loss: 0.8901
Epoch 8/100
3823/3823 - 4s - loss: 0.8904
Epoch 9/100
3823/3823 - 4s - loss: 0.8902
Epoch 10/100
3823/3823 - 4s - loss: 0.8904
Epoch 11/100
3823/3823 - 4s - loss: 0.8900
Epoch 12/100
3823/3823 - 4s - loss: 0.8896
Epoch 13/100
3823/3823 - 4s - loss: 0.8904
Epoch 14/100
3823/3823 - 4s - loss: 0.8900
Epoch 15/100
3823/3823 - 4s - loss: 0.8904
Epoch 16/100
3823/3823 - 4s - loss: 0.8903
Epoch 17/100
3823/3823 - 4s - loss: 0.8901
Epoch 18/100
3823/3823 - 4s - loss: 0.8898
Epoch 19/100
3823/3823 - 4s - loss: 0.8901
Epoch 20/100
3823/3823 - 4s - loss: 0.8903
Epoch 21/100
3823/3823 - 4s - loss: 0.8897
Epoch 22/100
3823/3823 - 4s - loss: 0.8897
Epoch 23/100
3823/3823 - 4s - loss: 0.8903
Epoch 24/100
3823/38

In [ ]:
# make predictions
trainPredict22 = model.predict(tensor_xtrain)
testPredict22 = model.predict(tensor_xtest)
ytrain = np.reshape(ytrain, (ytrain.shape[0], 1))
#calculate root mean squared error
trainScore22 = math.sqrt(mean_squared_error(ytrain, trainPredict22[:,0]))
print('Train Score: %.2f RMSE' % (trainScore22))
testScore22 = math.sqrt(mean_squared_error(ytest, testPredict22[:,0]))
print('Test Score: %.2f RMSE' % (testScore22))

Train Score: 0.94 RMSE
Test Score: 1.00 RMSE
